### Creating a test One Concern STAC catalog 

In [1]:
from pathlib import Path
import pystac
from pystac.extensions.projection import AssetProjectionExtension
import fsspec
import xarray as xr
import rasterio
import pandas as pd
import json

In [2]:
def get_bbox(ds):
    return ds.longitude.values.min(), ds.latitude.values.min(), ds.longitude.values.max(), ds.latitude.values.max()

In [19]:
jma_kerchunk_path = "gs://sam-temp-dev/kerchunk/jma_hist_rainfall_interpolated_nc_v2_consolidated_20060101_20190930.json"  # JMA
jwa_kerchunk_path = "gs://sam-temp-dev/kerchunk/jwa_historical_reanalysis_rainfall_netcdf_20191101_20201231.json"  # JWA, Mind the gap!!

ds_jma = xr.open_dataset(
    "reference://", engine="zarr",
    backend_kwargs={
        "storage_options": {
            "fo": jma_kerchunk_path,
            "remote_protocol": "gcs",
            "remote_options": {"anon": True}
        },
        "consolidated": False
    }
)

ds_jwa = xr.open_dataset(
    "reference://", engine="zarr",
    backend_kwargs={
        "storage_options": {
            "fo": jwa_kerchunk_path,
            "remote_protocol": "gcs",
            "remote_options": {"anon": True}
        },
        "consolidated": False
    }
)

ds_jwa

<xarray.Dataset>
Dimensions:    (latitude: 3360, longitude: 2560, time: 10248)
Coordinates:
  * latitude   (latitude) float64 20.0 20.01 20.02 20.03 ... 47.98 47.99 48.0
  * longitude  (longitude) float64 118.0 118.0 118.0 118.0 ... 150.0 150.0 150.0
  * time       (time) datetime64[ns] 2019-11-01 ... 2020-12-31T23:00:00
Data variables:
    rainrate   (time, latitude, longitude) float32 ...

In [12]:
def create_xarray_dataset_item(ds: xr.Dataset,
                               item_id: str,
                               item_title: str,
                               stac_extensions: list,
                               item_href: str,
                               asset_key: str,
                               asset_name: str,
                               asset_media_type: pystac.MediaType,
                               asset_epsg: int,
                               collection_name: str):
    
    bbox = get_bbox(ds)
    start_time = pd.to_datetime(ds.isel(time=0).time.values)
    end_time = pd.to_datetime(ds.isel(time=-1).time.values)
    href = item_href

    
    item = pystac.Item(id=item_id,
                       geometry=[],
                       datetime=None,
                       bbox=bbox,
                       properties={"institution": "One Concern",
                                    "title": item_title,
                                    "start_datetime": start_time.strftime('%Y-%m-%d %H:00'),
                                    "end_datetime": end_time.strftime('%Y-%m-%d %H:00')},
                       stac_extensions=['https://stac-extensions.github.io/projection/v1.0.0/schema.json'],
                       href=item_href)
    
    # Add asset
    item.add_asset(key=asset_key, asset=pystac.Asset(href=item_href,
                                                    title=asset_name,
                                                    media_type=asset_media_type)) 

    # extend the asset with projection extension
    asset_ext = AssetProjectionExtension.ext(item.assets[asset_key])
    asset_ext.epsg = asset_epsg
    asset_ext.shape = ds[asset_name].shape
    asset_ext.bbox = bbox
    asset_ext.transform = rasterio.transform.from_bounds(bbox[0], bbox[1], bbox[2], bbox[3], ds[asset_name].shape[1], ds[asset_name].shape[0])
    
    # if we add a collection we MUST add a link
    item.add_link(
        pystac.Link(
            pystac.RelType.COLLECTION,
            collection_name,
            media_type=asset_media_type,
        )
    )

    return item

In [13]:
%%time

# Create the initial
catalog = pystac.Catalog(
    id='One Concern STAC catalog',
    description='Various One Concern datasets supporting dev and prod operations',
    stac_extensions=["https://stac-extensions.github.io/projection/v1.0.0/schema.json"],
    href="gs://sam-temp-dev/stac",
    catalog_type=pystac.CatalogType.SELF_CONTAINED,
)

# Create an item for the weather data collection
weather_data_collection_name = "weather-data"

# Create an item to go into this collection
item_jma = create_xarray_dataset_item(ds=ds_jma,
                                      item_id="jma-historical-rainfall",
                                      item_title="JMA Interpolated Historical Rainfall (mm/hr)",
                                      stac_extensions=['https://stac-extensions.github.io/projection/v1.0.0/schema.json'],
                                      item_href=jma_kerchunk_path,
                                      asset_key="jma-rainrate",
                                      asset_name="rainrate",
                                      asset_media_type=pystac.MediaType.HDF5,
                                      asset_epsg=4326,
                                      collection_name=weather_data_collection_name)


# Create the collection using the total temporal and spatial extents of all items it contains
spatial_extent = pystac.SpatialExtent(bboxes=[item_jma.bbox])
jma_start_time = item_jma.properties["start_datetime"]
jma_end_time = item_jma.properties["end_datetime"]

temporal_extent = pystac.TemporalExtent([[jma_start_time, jma_end_time]])
collection_extent = pystac.Extent(spatial=spatial_extent, temporal=temporal_extent)
collection = pystac.Collection(id=weather_data_collection_name,
                               description="Historic rainfall and meteorological data from various sources (JMA, JWA, etc)",
                               extent=collection_extent,
                               license="One Concern, Inc")

# Add item(s) to the collection
collection.add_item(item_jma)

# Add collection to the catalog
catalog.add_child(collection)

CPU times: user 3.11 ms, sys: 1.59 ms, total: 4.71 ms
Wall time: 3.27 ms


In [15]:
# catalog

In [ ]:
# catalog.make_all_asset_hrefs_relative()
catalog.normalize_hrefs("gs://sam-temp-dev/stac/one-concern-stac-catalog")
catalog.get_self_href()

In [ ]:
catalog

### Write the catalog directly to GCS

In [ ]:
%%time
import gcsfs
fs = gcsfs.GCSFileSystem()  # project='onec-dev'
indent=2

# Write catalog
with fs.open("gs://sam-temp-dev/stac/one-concern-stac-catalog/catalog.json", "w", content_type="application/json") as f:
    f.write(json.dumps(catalog.to_dict(), indent=indent))

# Write collection
with fs.open("gs://sam-temp-dev/stac/one-concern-stac-catalog/weather-data/collection.json", "w", content_type="application/json") as f:
    f.write(json.dumps(collection.to_dict(), indent=indent))

# Write item(s)
with fs.open("gs://sam-temp-dev/stac/one-concern-stac-catalog/weather-data/jwa-historical-rainfall/jwa-historical-rainfall.json", "w", content_type="application/json") as f:
    f.write(json.dumps(item.to_dict(), indent=indent))

### To read the catalog from GCS, we need to re-define the StacIO class to use fsspec
(taken from stactools: https://stactools.readthedocs.io/en/latest/_modules/stactools/core/io.html#FsspecStacIO)

In [ ]:
from pystac.stac_io import DefaultStacIO, StacIO

class FsspecStacIO(DefaultStacIO):
    """A subclass of :py:class:`pystac.DefaultStacIO` that uses `fsspec
    <https://filesystem-spec.readthedocs.io/en/latest/>`_ for reads and writes.
    """

    def read_text_from_href(self, href: str) -> str:
        """Reads a file as a utf-8 string using `fsspec
        <https://filesystem-spec.readthedocs.io/en/latest/>`_

        Args:
            href (str): The href to read.

        Returns:
            str: The read text, decoded as utf-8 if necessary.
        """ 
        with fsspec.open(href, "r") as f:
            s = f.read()
            if isinstance(s, str):
                return s
            elif isinstance(s, bytes):
                return str(s, encoding="utf-8")
            else:
                raise ValueError(f"Unable to decode data loaded from HREF: {href}")
                
StacIO.set_default(FsspecStacIO)

In [ ]:
pystac.Catalog.from_file("gs://sam-temp-dev/stac/one-concern-stac-catalog/catalog.json")  #, stac_io=FsspecStacIO)

### Other testing is below here

In [ ]:
with fsspec.open("gs://sam-temp-dev/stac/one-concern-stac-catalog/catalog.json") as file: 
    data = json.load(file)
    
cat_gcs = pystac.Catalog.from_dict(data) # , root="https://storage.cloud.google.com/sam-temp-dev/stac/one-concern-stac-catalog/catalog.json")
cat_gcs

In [ ]:
collections = list(catalog.get_collections())

In [ ]:
for collection in collections:
    print(collection)

In [ ]:
collection = catalog.get_child("weather-data")
collection

In [ ]:
for root, subcategories, items in catalog.walk():

    print('{} is the current root in the STAC'.format(root.id))
    print('Are there any any STAC validation errors ?: {}'.format(root.validate()))
    
    for subcat in subcategories:
        print('{} is a subcatalog or collection for the current root in the STAC'.format(subcat.id))
        print('Are there any STAC validation errors ?: {}'.format(subcat.validate()))
        
    for item in items:
        print('{} is a STAC Item within the current root'.format(item.id))
        print('Are there any STAC validation errors ?: {}'.format(item.validate()))

### GEE STAC Catalog

In [ ]:
gee_cat_path = "https://earthengine-stac.storage.googleapis.com/catalog/catalog.json"

In [ ]:
sen2_level2a = "https://storage.googleapis.com/earthengine-stac/catalog/COPERNICUS/COPERNICUS_S2_CLOUD_PROBABILITY.json"

In [ ]:
stac_url = (
    "https://earthengine-stac.storage.googleapis.com/catalog/catalog.json"  # https://earthengine-stac.storage.googleapis.com/catalog/catalog.json
)

stac_url = "https://storage.googleapis.com/earthengine-stac/catalog/COPERNICUS/catalog.json"

stac_url = 'https://storage.googleapis.com/earthengine-stac/catalog/COPERNICUS/COPERNICUS_S1_GRD.json'

with urllib.request.urlopen(stac_url) as url:
    data = json.loads(url.read().decode())


In [ ]:
data

In [ ]:
for link in data["links"]:
    print(link)
    try:
        if link["title"] == "COPERNICUS_S1_GRD":
            href = link["href"]
            break
    except:
        pass
    # break

In [ ]:
href

In [ ]:
stac_url = "https://earth-search.aws.element84.com/v0"
catalog = pystac_client.Client.open(stac_url)
catalog

In [ ]:
collections = list(catalog.get_collections())
for collection in collections:
    print(collection)

In [ ]:
collection = catalog.get_child("sentinel-s2-l1c")
collection

In [ ]:
bbox = [139.4038110856679964, 38.1097418227999967, 139.4887555307919911, 38.1482418231080018]
datetime = "2021-08-01/2022-09-10"

# bbox = [xmin, ymin, xmax, ymax]
# catalog = pystac_client.Client.open(
#     "https://planetarycomputer.microsoft.com/api/stac/v1"
# )
search = catalog.search(
    collections=["sentinel-s2-l1c"], bbox=bbox, datetime=datetime)

In [ ]:
items = search.item_collection()
print(f"Found {len(items)} items")

## PlanetaryComputer Interactions

In [ ]:
import os
import urllib

import requests
import pystac
import pystac_client
from pystac.extensions.eo import EOExtension as eo
import planetary_computer
import rasterio
import stackstac
import hvplot.xarray

# import dask.diagnostics
# with dask.diagnostics.ProgressBar():
#     data = aoi.compute()

In [ ]:
# IF NEEDED, generate a token

# BASE_TOKEN_URL = "https://planetarycomputer.microsoft.com/api/sas/v1/token"

# collection = "sentinel-2-l2a"

# url = os.path.join(BASE_TOKEN_URL, collection)

# headers = {'accept': 'application/json'}
# response = requests.get(url, headers=headers)
# response.json()

In [ ]:
%%time
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)
collections = list(catalog.get_collections())
collections

In [ ]:
# # Murakami
# bbox = [139.4038110856679964, 38.1097418227999967, 139.4887555307919911, 38.1482418231080018]
# datetime = "2022-08-01/2022-08-04"

bbox = [-82.58766624, 25.56165135, -80.69396004, 27.16010966]  # SW Florida

# bbox = [xmin, ymin, xmax, ymax]
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1"
)
search = catalog.search(
    collections=["3dep-seamless"], bbox=bbox   #  datetime=datetime
)
items = search.item_collection()
print(f"Found {len(items)} items")

In [ ]:
for item in items:
    print(item)

In [ ]:
%env PC_SDK_SUBSCRIPTION_KEY=""  # did this work or was it the command line approach??

In [ ]:
item_signed = planetary_computer.sign(item)

In [ ]:
ds = stackstac.stack(item_signed)

In [ ]:
vv = ds.sel(band="vh").compute()

In [ ]:
vv

In [ ]:
ds.sel(band="vh").hvplot.quadmesh(x='x',
                                   y='y',
                                   # title='dis',
                                   # geo=True,
                                   width=750,
                                   height=700,
                                   rasterize=True,
                                   project=True,
                                   cmap="bmw",
                                   # clim=(50, 100),
                                   tiles='EsriImagery')

In [ ]:
collection = catalog.get_child("sentinel-1-rtc")  # Need token to access!

In [ ]:
BASE_TOKEN_URL = "https://planetarycomputer.microsoft.com/api/sas/v1/token"

collection = "sentinel-1-rtc"

key = ""
collection_url = collection + "?subscription-key=" + key

url = os.path.join(BASE_TOKEN_URL, collection_url)
url

headers = {'accept': 'application/json'}
response = requests.get(url, headers=headers)
token = response.json()["token"]
token

In [ ]:
sen1_rtc_url= "https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-1-rtc"
sen1_rtc_url_token = sen1_rtc_url + "?" + token
sen1_rtc_url_token

In [ ]:
response = requests.get(sen1_rtc_url)
collection_json = response.json()
collection_json

In [ ]:
pystac.Collection(collection_json)

In [ ]:
search = pystac_client.ItemSearch(
    url=sen1_rtc_url_token,
    bbox=bbox_of_interest,
    datetime=time_of_interest,
)

# # Check how many items were returned
# items = search.item_collection()
# print(f"Returned {len(items)} Items")

In [ ]:
for item in search.items():
    print(item.id)
    break

In [ ]:
# Murakami City, JP
bbox_of_interest = [139.4038110856679964, 38.1097418227999967, 139.4887555307919911, 38.1482418231080018]

time_of_interest = "2022-08-01/2022-08-06"

In [ ]:
search = catalog.search(
    collections=["sentinel-1-rtc"],
    bbox=bbox_of_interest,
    datetime=time_of_interest,
)

# Check how many items were returned
items = search.item_collection()
print(f"Returned {len(items)} Items")

In [ ]:
search = catalog.search(
    collections=["sentinel-2-l2a"],
    # intersects=area_of_interest,
    bbox=bbox_of_interest,
    datetime=time_of_interest,
    query={"eo:cloud_cover": {"lt": 90}},
)

# Check how many items were returned
items = search.item_collection()
print(f"Returned {len(items)} Items")

In [ ]:
item = items[0]
item

In [ ]:
blue_href = item.assets["B02"].href
green_href = item.assets["B03"].href
swir_href = item.assets["B11"].href

In [ ]:
item.assets

In [ ]:
%%time
with rasterio.open(blue_href) as ds:
    profile = ds.profile.copy()
    arr_blue = ds.read(1)
profile

In [ ]:
arr_blue.shape

In [ ]:
arr_blue